In [ ]:
import numpy as np
import torch 
from torch import nn
from PIL import Image
import matplotlib.pyplot as plt
import os
from torchvision import datasets, transforms,utils
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.5],std=[0.5])])

# 数据集载入
train_data = datasets.MNIST(root = "./data/",transform=transform,train = True,download = True)
test_data = datasets.MNIST(root="./data/",transform = transform,train = False)
print('训练集/测试集:',len(train_data),'/',len(test_data))

#加载器
train_loader = torch.utils.data.DataLoader(train_data,batch_size=32,shuffle=True,num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=32,shuffle=True,num_workers=2)
print('训练batch/测试batch:',len(train_loader),'/',len(test_loader))



In [ ]:
data_iter = iter(train_loader)
nd = next(data_iter)
oneimg,label = nd[0][0],nd[1][0]
grid = utils.make_grid(oneimg)
grid = grid.numpy().transpose(1,2,0) 
std = [0.5]
mean = [0.5]
grid = grid * std + mean
plt.figure(figsize=(2,2))
plt.imshow(grid)
plt.show()
print(label)

images, lables =nd
plt.figure(figsize=(12,4))
img = utils.make_grid(images,32)
# transpose 转置函数(x=0,y=1,z=2),新的x是原来的y轴大小，新的y是原来的z轴大小，新的z是原来的x大小
#相当于把x=1这个一道最后面去。
imgs = img.numpy().transpose(1,2,0) 
plt.imshow(imgs)
plt.show()
print(' ','  '.join('%d'%i for i in lables))


In [ ]:
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1)
        self.pool = nn.MaxPool2d(2,2)
        
        self.conv2 = nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1)
        self.fh0 = nn.Linear(64*7*7,1024*3)#两个池化，所以是7*7而不是14*14
#         self.fh1 = nn.Linear(1024*3,1024)
        self.fh2 = nn.Linear(1024*3,256)
        
        self.fo = nn.Linear(256,10)
        self.dp = nn.Dropout(p=0.2)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, 64 * 7* 7)#将数据平整为一维的 
        x = self.dp(x)
        x = F.relu(self.fh0(x))
        x = self.dp(x)
#         x = F.relu(self.fh1(x))
        x = F.relu(self.fh2(x))  
        x = self.dp(x)
        x = self.fo(x)  
#         x = F.log_softmax(x,dim=1) NLLLoss()才需要，交叉熵不需要
        return x

def draw_train_process(title,iters,costs,accs,label_cost,lable_acc):
    fig, axes = plt.subplots(1, 2, figsize=[16, 5])
    sns.lineplot(iters, costs,color='red',label=label_cost, ax = axes[0]) 
    sns.lineplot(iters, accs,color='green',label=lable_acc,ax=axes[1]) 
    plt.show()
# train_iters = range(len(train_accs))

In [ ]:
import datetime as dt
import tqdm.notebook as tqnb
tstart=dt.datetime.now()
train_accs = []
train_loss = []
torch.cuda.empty_cache()
net = CNN()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adagrad(net.parameters(), lr=0.001)
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
#optimizer = torch.optim.Adam(net.parameters(),lr=0.001)#Adam优化方法比较时髦

roundx=1
pbar = tqnb.tqdm(total=len(train_loader)*roundx)
for epoch in range(roundx):
    running_loss = 0.0
    for i,data in enumerate(train_loader,0):#0是下标起始位置默认为0
        # data 的格式[[inputs, labels]]       
#         inputs,labels = data
        inputs,labels = data[0].to(device), data[1].to(device)
        #初始为0，清除上个batch的梯度信息
        optimizer.zero_grad()         

        #前向+后向+优化     
        outputs = net(inputs)
        print(outputs,labels) 
        break;
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        # loss 的输出，每个一百个batch输出，平均的loss
        running_loss += loss.item()
#         if i%100 == 99:
#             print('%d,%d-%.3f' % (epoch+1,i+1,running_loss/100), end='|')
#             running_loss = 0.0
        train_loss.append(loss.item())

        # 训练曲线的绘制 一个batch中的准确率
        correct = 0
        total = 0
        _, predicted = torch.max(outputs.data, 1)
        total = labels.size(0)# labels 的长度
        correct = (predicted == labels).sum().item() # 预测正确的数目
        train_accs.append(100*correct/total)
        pbar.update(1)
pbar.close()
torch.cuda.empty_cache()
dt.datetime.now()-tstart

import seaborn as sns

draw_train_process('training',train_iters,train_loss,train_accs,'training loss','training acc')
print(train_accs[-20:])
print(dt.datetime.now()-tstart)


In [ ]:
dataiter = iter(test_loader)
images,labels = dataiter.next()




#device='cpu'
net.to(device)

from sklearn.metrics import classification_report

l=[]
p=[]
# pbar = tqnb.tqdm(total=len(test_loader))

for i,data in  enumerate(test_loader,0):
    inputs,labels = data[0].to(device), data[1]
    l.extend(labels)
    test_out = net(inputs)
    predicted = torch.max(test_out, dim=1)[1]
    p.extend(predicted.cpu())
    x=  predicted.cpu()-labels
    if len(x[x != 0]):
        plt.figure(figsize=(12,4))
        plt.xticks([])  #去掉x轴
        plt.yticks([])  #去掉y轴
        plt.axis('off')  #去掉坐标轴
        test_img = utils.make_grid(inputs.cpu(),32)
        test_img = test_img.numpy().transpose(1,2,0)
        test_img[test_img < 0] = 0
        plt.imshow(test_img)
        plt.show()
        ls=[]
        ps=[]
        for i in range(len(x)):
            if x[i] == 0 :
                ls.append('-')
                ps.append('-')
            else:
                ls.append('%d'%int(labels[i]))
                ps.append('%d'%int(predicted[i]))
                          
        print('R', '  '.join(j for j in ls))
        print('G', '  '.join(j for j in ps))

print('算法评价:')
print(classification_report(l, p,digits=5))



In [ ]:
torch.save(net.state_dict(), "./ai_991")